In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore, auth
from datetime import datetime
from dotenv import load_dotenv
import os
import json

load_dotenv()

cred = credentials.Certificate(os.getenv("firebase_secret"))
app = firebase_admin.initialize_app(cred)


db = firestore.client()


In [22]:
salas_destino = {
  "DET": ["B303", "B304", "B305", "B306"],
  "MAT": ["B203", "B204", "B205", "B206"],
  "EMT": ["B319", "B320", "B214"],
  "CEE": ["B308", "B309", "B310", "B321", "B322", "B323"],
  "CAC": ["B207", "B208", "B209", "B210", "B307"]
}


In [3]:
import pandas as pd

In [4]:
professores = pd.read_html("https://servidores.blumenau.ufsc.br/servidores-docentes/")

In [5]:
import requests 
pagecontent = requests.get("https://servidores.blumenau.ufsc.br/servidores-docentes/").text


In [6]:

import re

def extract_text(html_string):
    pattern = r'<h3>(.*?)</h3>'
    match = re.findall(pattern, html_string, re.MULTILINE)
    if match is None:
        return ''
    return [x for x in match]

def extract_regex_group(html_string, pattern): 
    match = re.finditer(pattern, html_string, re.MULTILINE)
    if match is None:
        return ''
    return [x.group(1) for x in match]


In [7]:
ordem_tabelas = [extract_regex_group(x, r"\((.*?)\)")[0] for x in  extract_text(pagecontent)]

In [8]:
for i in range(len(ordem_tabelas)):
    professores[i]["Departamento"] = ordem_tabelas[i]

In [9]:
professores = pd.concat(professores)

In [16]:
salas_origem = {}
for departamento in ordem_tabelas:
    lista_salas = professores[professores["Departamento"] == departamento]["Sala"].unique().tolist()
    salas_C = [ x for x in lista_salas if type(x) == str and x.startswith("C")]
    salas_origem[departamento] = salas_C

In [24]:

for key, val in salas_destino.items():
    val.sort()

for key, val in salas_origem.items():
    val.sort()


In [25]:
salas_destino, salas_origem

({'DET': ['B303', 'B304', 'B305', 'B306'],
  'MAT': ['B203', 'B204', 'B205', 'B206'],
  'EMT': ['B214', 'B319', 'B320'],
  'CEE': ['B308', 'B309', 'B310', 'B321', 'B322', 'B323'],
  'CAC': ['B201', 'B207', 'B208', 'B209', 'B307']},
 {'EMT': ['C101'],
  'CEE': ['C102', 'C201', 'C203', 'C204', 'C205'],
  'CAC': ['C102', 'C202', 'C303', 'C304', 'C306'],
  'DET': ['C204', 'C302', 'C305'],
  'MAT': ['C202', 'C204', 'C301', 'C306']})

# Atualizando DB

In [53]:
from pathlib import Path

ambientes_norte = Path("json0929/ambientes-norte.json")

with open(ambientes_norte) as f:
    ambientes_norte = json.load(f)

In [51]:
def get_sala(codigo):
    return next(filter(lambda x: x["codigo"] == codigo, ambientes_norte))

In [56]:
for key, val in salas_destino.items():
    origem = salas_origem[key]
    print("-"*5, key, "-"*5)
    for codigo in val:
        ambiente = get_sala(codigo) 
        print(codigo, ":", ambiente["nome"])
        #print({"nome": f"Sala professores {key}", "origem": origem})

----- DET -----
B303 : Sala de Professores 10
B304 : Sala de Professores 11
B305 : Sala de Professores 12
B306 : Sala de Professores 13
----- MAT -----
B203 : Sala de Professores 01
B204 : Sala de Professores 02
B205 : Sala de Professores 03
B206 : Sala de Professores 04
----- EMT -----
B214 : Sala de Professores 09
B319 : Sala de Professores 18
B320 : Sala de Professores 19
----- CEE -----
B308 : Sala de Professores 15
B309 : Sala de Professores 16
B310 : Sala de Professores 17
B321 : Sala de Professores 20
B322 : Sala de Professores 21
B323 : Sala de Professores 22
----- CAC -----
B201 : Laboratórios de Informática I (LABINFO II)
B207 : Sala de Professores 05
B208 : Sala de Professores 06
B209 : Sala de Professores 07
B307 : Sala de Professores 14


In [50]:

# testing 
bababase = [{
    "codigo": "A",
    "nome": "Sala professores DET",
}, {
    "codigo": "B",
    "nome": "Sala professores MAT",
}]

def get_sala_test(codigo):
    return next(filter(lambda x: x["codigo"] == codigo, bababase))

test1 = get_sala_test("A")
test1["nome"] = "Sala Qualquer"
bababase


[{'codigo': 'A', 'nome': 'Sala Qualquer'},
 {'codigo': 'B', 'nome': 'Sala professores MAT'}]